In [110]:
import joblib
from utils.process_images import *  # Import your image processing functions
from utils.process_data import *  # Import your data processing functions

In [111]:
# Step 1: Load the saved model
model_filename = 'rf_model_wave.joblib'  # Replace with the correct filename
loaded_model = joblib.load(model_filename)


In [117]:
# Step 2: Process a new image to create the feature NumPy arraydrawings
new_image_path = 'drawings\\wave\\training\\parkinson\\V04PO02.png'  

# Define a function for processing a new image and making predictions
def process_new_image(new_image_path):
    new_image = read_and_thresh(new_image_path, resize=False)

    # Compute the same interaction features as during training
    mean_thickness = np.mean(stroke_thickness(closing(label_sort(new_image) > 0, disk(1))))
    std_thickness = np.std(stroke_thickness(closing(label_sort(new_image) > 0, disk(1))))
    num_pixels = sum_pixels(skeleton_drawing(new_image))
    num_ep = number_of_end_points(new_image, k_nn)
    num_inters = number_of_intersection_points(new_image, k_nn)

    # Create a feature array matching the training data
    new_features = np.array([
        mean_thickness, std_thickness, num_pixels, num_ep, num_inters,
        mean_thickness * std_thickness, mean_thickness * num_pixels, mean_thickness * num_ep, mean_thickness * num_inters,
        std_thickness * num_pixels, std_thickness * num_ep, std_thickness * num_inters,
        num_pixels * num_ep, num_pixels * num_inters, num_ep * num_inters
    ]).reshape(1, -1)

    return new_features
new_features = process_new_image(new_image_path)

In [118]:
# Step 3: Use the loaded model to make predictions
predicted_probabilities = loaded_model.predict_proba(new_features)

if predicted_probabilities[0][1] < 0.87:
    print('healthy')
else :
    print('parkinson')


parkinson


c:\Users\Taqana\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
